# Thesis Audit Template — Cyberhate Detection
Pipeline: **Train → Calibrate → Fairness → Functional → Robustness → Faithfulness → Risk–Coverage**.

In [ ]:
import os, subprocess, numpy as np, pandas as pd
run_dir = 'runs/roberta_bin'  # set to your trained run
print('Using run_dir=', run_dir)


## 1. Calibration

In [ ]:
cmd = ['python','scripts/calibrate.py','--logits',f'{run_dir}/val_logits.npy','--labels',f'{run_dir}/val_labels.npy','--out',f'{run_dir}/val_probs_cal.npy']
print(' '.join(cmd)); subprocess.run(cmd, check=True)


## 2. Fairness (BPSN/BNSP/Subgroup AUC)

In [ ]:
preds_csv = 'eval/preds_with_identities.csv'  # prepare externally
cmd = ['python','scripts/eval.py','--preds',preds_csv,'--id-cols','identity_women,identity_muslim,identity_black']
print(' '.join(cmd)); subprocess.run(cmd, check=True)


## 3. Functional Tests (HateCheck)

In [ ]:
cmd = ['python','scripts/run_hatecheck.py','--model-path',run_dir,'--hatecheck','data/hatecheck.csv']
print(' '.join(cmd)); subprocess.run(cmd, check=True)


## 4. Robustness: OCR/ASR Noise Sweep (prepare noisy eval CSV)

In [ ]:
from cyberhate_lab.ocr_asr.noise_simulation import batch_corrupt
df = pd.read_csv('eval/eval_texts.csv')  # id,text,label
df['text_noisy'] = batch_corrupt(df['text'].astype(str).tolist(), p_del=0.03, p_sub=0.03, p_ins=0.03)
df.to_csv('eval/eval_texts_noisy.csv', index=False)
print('Wrote eval/eval_texts_noisy.csv')


## 5. Faithfulness: Train spans then evaluate

In [ ]:
# Train spans externally: python -m cyberhate_lab.xai.train_spans --train data/spans_train.csv --val data/spans_val.csv --model roberta-base --out runs/span_model
cmd = ['python','scripts/eval_rationales.py','--model-path',run_dir,'--val-csv','data/spans_val.csv','--rationales-npy','runs/span_model/val_token_rationales.npy']
print(' '.join(cmd))
try:
    subprocess.run(cmd, check=True)
except Exception as e:
    print('Run span training first; then re-run this cell.')


## 6. Risk–Coverage

In [ ]:
cmd = ['python','scripts/risk_coverage.py','--probs',f'{run_dir}/val_probs.npy','--labels',f'{run_dir}/val_labels.npy','--out',f'{run_dir}/risk_coverage.png']
print(' '.join(cmd)); subprocess.run(cmd, check=True)
